# Word Count in Spark
We can find a cleaned-up text file with all of Shakespeare's work in

In [1]:
%%sh
hdfs dfs -ls /data/shakespeare

Found 1 items
-rw-r--r--   3 pmolnar hdfs    5308225 2017-10-19 16:50 /data/shakespeare/shakespeare.txt


In [2]:
DATADIR='/data/shakespeare'

In [ ]:
##sc.stop; del(sc)

In [3]:
# %load pyspark_init_arc.py
#
# This configuration works for Spark on macOS using homebrew
#
import os, sys
# set OS environment variable
os.environ["SPARK_HOME"] = '/usr/hdp/2.4.2.0-258/spark'
# add Spark library to Python
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], 'python'))

# import package
import pyspark
from pyspark.context import SparkContext, SparkConf

import atexit
def stop_my_spark():
    sc.stop()
    del(sc)

# Register exit    
atexit.register(stop_my_spark)

# Configure and start Spark ... but only once.
if not 'sc' in globals():
    conf = SparkConf()
    conf.setAppName('MyFirstSpark') ## you may want to change this
    conf.setMaster('yarn-client')   ##conf.setMaster('local[2]')
    sc = SparkContext(conf=conf)
    print "Launched Spark version %s with ID %s" % (sc.version, sc.applicationId)

Launched Spark version 1.6.1 with ID application_1508160140652_0002


In [4]:
print "http://arc.insight.gsu.edu:8088/cluster/app/%s"% (sc.applicationId)

http://arc.insight.gsu.edu:8088/cluster/app/application_1508160140652_0002


# Load Data

In [5]:
rdd = sc.textFile(os.path.join(DATADIR, 'shakespeare.txt')).sample(False, 0.001)

In [6]:
rdd.take(10)

[u"  Which in my bosom's shop is hanging still,",
 u'',
 u'  Thine eyes, that taught the dumb on high to sing,',
 u'    This is a dreadful sentence.',
 u'  BERTRAM. Do you think I am so far deceived in him?',
 u'    Ours be your patience then, and yours our parts;',
 u'    No more do yours. Your virtues, gentle master,',
 u'    how to know a man in love; in which cage of rushes I am sure you',
 u'',
 u"  By computation and mine host's report"]

# Cleaning-up

The `mapper.sh` code run some character replacements
<pre>
tr -d '.,:?"' \
| tr '[]{}-' '     ' \
| tr 'A-Z' 'a-z' \
| tr ' ' '\n' \
| grep -v -e '^[[:space:]]*$'

</pre>

We're going to use the regular expression package, `re` to replace characters.
<pre>
regex = re.compile('[%s]' % re.escape(string.punctuation))
regex.sub(' ', s)
</pre>


In [7]:
import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))

In [8]:
print "Special characters: %s"%(string.punctuation)

Special characters: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [9]:
# hint: regex.sub(' ', s)
rdd2 = rdd.map(lambda s: regex.sub(' ', s))
rdd2.take(4)

[u'  Which in my bosom s shop is hanging still ',
 u'',
 u'  Thine eyes  that taught the dumb on high to sing ',
 u'    This is a dreadful sentence ']

Let's convert to lower case

In [10]:
rdd3 = rdd2.map(lambda s: s.lower())
rdd3.take(4)

[u'  which in my bosom s shop is hanging still ',
 u'',
 u'  thine eyes  that taught the dumb on high to sing ',
 u'    this is a dreadful sentence ']

`map()` vs `flatMap()`
- `map` produces a single row per row, even if the row may contain a collection
- `flatMap` if the function on the row produces a collection multiple rows will be ejected

In [11]:
# hint: s.split(' ')
rdd4 = rdd3.flatMap(lambda s: s.split(' '))
rdd4.take(4)

[u'', u'', u'which', u'in']

Now, that we have the words extracted we still need to add a value for the reduce process $x \rightarrow (x,1)$

In [12]:
rdd5 = rdd4.map(lambda t: (t, 1))
rdd5.take(4)

[(u'', 1), (u'', 1), (u'which', 1), (u'in', 1)]

#  Counting
Now, our data set should be in the proper format, and we can count the words

In [13]:
# hint: +
wordcount = rdd5.reduceByKey(lambda a,b: a+b)
wordcount.take(4)

[(u'', 995), (u'shop', 1), (u'all', 3), (u'words', 1)]

Let's also sort them in descending order ... may have to swap values within the row

In [14]:
wordcount.map(lambda t: (t[1], t[0])).sortByKey(False).take(10)

[(995, u''),
 (33, u'and'),
 (31, u'the'),
 (24, u'a'),
 (21, u'i'),
 (19, u'of'),
 (17, u'to'),
 (15, u'in'),
 (14, u'my'),
 (14, u'you')]